# Predicting the past, United States

## Introduction

In the United States, 112 commercial nuclear reactors have operated since 1967. Published data of these reactors is used in _CYCLUS_, an agent-based simulation software, in order to compare simulation results with real world metrics and produce  baseline comparison models for future simulations. Information regarding reactors are obtained from Power Reactor Information System's (PRIS) reactor database. [reactors_pris_2016.csv](import_data/reactors_pris_2016.csv)

## Obtaining Recipes for Simulation

First, an input file for _CYCLUS_ specifying the recipes for fresh and spent nuclear fuel (SNF) is produced. The recipes for fresh and SNF are obtained from [vision_recipes.xls](import_data/vision_recipies/vision_recipies.xls) spreadsheet file, which contains isotopes and compositions for fresh and SNF for uranium oxide (UOX) and mixed oxide (MOX) fuel at different burn ups.

Importing raw composition data from `vision_recipes.xls` is performed with a python script and is demonstrated below. Due to limited information about the reactors, all reactors are assumed to operate at a burn up of 51 GWd/MTHM.

### Demonstration
__Import necessary libraries__

In [3]:
import csv
import collections
import dateutil.parser as date
import jinja2
import matplotlib.pyplot as plt
import numpy as np
import os
import pathlib
import sys
import sqlite3 as lite
from matplotlib import cm
from pyne import nucname as nn
from IPython.display import Image

sys.path.append('../../scripts/')
import analysis as an

region = 'united_states'

__Read fuel recipe data__

`vision_recipes.xls` is stored to a list object by looping over each row in the spreadsheet file.

In [4]:
def import_csv(in_csv, delimit):
    """ Imports contents of a csv text file to a list of
    lists.

    Parameters
    ---------
    in_csv: str
        path and name of input csv file
    delimit: str
        delimiter of the csv file

    Returns
    -------
    data_list: list
        list of lists containing the csv data
    """
    with open(in_csv, encoding='utf-8') as source:
        sourcereader = csv.reader(source, delimiter=delimit)
        data_list = []
        for row in sourcereader:
            data_list.append(row)
    return data_list


recipes = import_csv('../../database/vision_recipes/uox.csv', ',')
print(*recipes[4:8], '.', '.', '.', sep='\n')

__Load template for _CYCLUS_ input file__

A jinja template file, used to produce parts of the final _CYCLUS_ simulation input file, is loaded to memory.

In [5]:
def load_template(in_template):
    """ Returns a jinja2 template from file.

    Parameters
    ---------
    in_template: str
        path and name of jinja2 template

    Returns
    -------
    output_template: jinja template object
    """
    with open(in_template, 'r') as default:
        output_template = jinja2.Template(default.read())
    return output_template


recipe_template = load_template('templates/recipes_template.xml')

__Select appropriate recipe for a given burnup__

The appropriate composition for fresh and spent nuclear fuel is selected by specifying burnup. The composition data is then stored into a dictionary object with {isotope: mass fraction composition} as {key : value} pair.

In [9]:
def get_composition_fresh(in_list, burnup):
    """ Returns a dictionary of isotope and composition (in mass fraction)
    using vision_recipes for fresh UOX fuel.

    Parameters
    ---------
    in_list: list
        list containing vision_recipes
    burnup: int
        burnup

    Returns
    -------
    data_dict: dict
        dictionary with key=[isotope],
        and value=[composition]
    """
    data_dict = {}
    for i in range(len(in_list)):
        if i > 1:
            if burnup == 33:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][1])})
            elif burnup == 51:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][3])})
            else:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][5])})
    return data_dict


def get_composition_spent(in_list, burnup):
    """ Returns a dictionary of isotope and composition (in mass fraction)
    using vision_recipes for spent nuclear fuel

    Parameters
    ---------
    in_list: list
        list containing vision_recipes data
    burnup: int
        burnup

    Returns
    -------
    data_dict: dict
        dictionary with key=[isotope],
        and value=[composition]
    """
    data_dict = {}
    for i in range(len(in_list)):
        if i > 1:
            if burnup == 33:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][2])})
            elif burnup == 51:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][4])})
            else:
                data_dict.update({nn.id(in_list[i][0]):
                                  float(in_list[i][6])})
    return data_dict


fresh33 = get_composition_fresh(recipes, 33)
spent33 = get_composition_spent(recipes, 33)
fresh51 = get_composition_fresh(recipes, 51)
spent51 = get_composition_spent(recipes, 51)
fresh100 = get_composition_fresh(recipes, 100)
spent100 = get_composition_spent(recipes, 100)
print('Spent Fuel Composition:')
print('ISOTOPE:   Massfrac')
for k in list(spent51.keys())[:3]:
    print(k, ': ', spent51[k])
print('.', '.', '.', sep='\n')

__Render recipe template with recipe data__

`jinja2` library is used to render recipe data to the previously loaded `recipe_template`. `pyne` library is used to convert the isotope names to _CYCLUS_ readable format. The rendered information is then saved to an xml file for use with _CYCLUS_.

In [10]:
def write_recipes(in_template, burnup, region):
    """ Renders jinja template using fresh and spent fuel composition.

    Parameters
    ---------
    fresh_dict: dict
        dictionary with key=[isotope], and
        value=[composition] for fresh UOX
    spent_dict: dict
        dictionary with key=[isotope], and
        value=[composition] for spent fuel
    in_template: jinja template object
        jinja template object to be rendered
    burnup: int
        amount of burnup

    Returns
    -------
    null
        generates recipe files for cyclus.
    """
    fresh_dict = get_composition_fresh(recipes, burnup)
    spent_dict = get_composition_spent(recipes, burnup)
    out_path = 'cyclus/input/' + region + '/recipes/'
    pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)
    rendered = in_template.render(fresh=fresh_dict,
                                  spent=spent_dict)
    with open(out_path + '/uox_' + str(burnup) + '.xml', 'w') as output:
        output.write(rendered)


write_recipes(fresh33, spent33, recipe_template, 33, region)
write_recipes(fresh51, spent51, recipe_template, 51, region)
write_recipes(fresh51, spent51, recipe_template, 100, region)

[uox_51.xml](cyclus/input/united_states/recipes/uox_51.xml) is a sample rendered result that contains fresh and spent nuclear fuel compositions. 

__recipe template__

`jinja2` library is used to produce various parts of the final  _CYCLUS_ input file using templates. The library replaces simulation-specific information, such as the recipe for SNF, from the specified template file. This allows end-users to produce _CYCLUS_ simulation inputs that vary depending on the data with which they have been rendered. An example is shown below.

Recipe template file:

```
    <recipes>
      <recipe>
        <name>fresh_uox</name>
        <basis>atom</basis>
        <nuclide>
          <id>922350000</id>
          <comp>4.5</comp>
        </nuclide>
        <nuclide>
          <id>922380000</id>
          <comp>95.5</comp>
        </nuclide>
      </recipe>
      <recipe>
        <name>spent_uox</name>
        <basis>atom</basis>
        {% for key, value in vision.items() -%}
        <nuclide>  <id>{{ key }}</id>  <comp>{{ value }}</comp>  </nuclide>
        {% endfor -%}
      </recipe>
    </recipes>
```

The template shown above is a recipe template. `jinja2` recognizes variables within the brackets (`{{ }}`) and replaces the variables with appropriate values. The library also recognizes loops to produce multiple lines of text from a list of data. For instance, `jinja2` used the for-loop in the following lines
```
    {% for key, value in vision.items() -%}`
    <nuclide>  <id>{{ key }}</id>  <comp>{{ value }}</comp>  </nuclide>
    {% endfor -%}
```
and iterated over the composition dictionary to render the isotope and its composition.

[uox_51.xml](cyclus/input/US/recipes/uox_51.xml) is the rendered result. 

## Obtaining Deployment Data

Reactors specified in [reactors_pris_2016.csv](import_data/reactors_pris_2016.csv), need to be properly imported. The reactors should be deployed at the correct timesteps for an accurate simulation. The spreadsheet file contains reactor information including the name, deployment date, net capacity, and deployment nation. Obtaining reactor information for _CYCLUS_ is performed using a set of python functions. The same generic steps were repeated for this demonstration: importing data stored in a delimited text file, and rendering the imported data to a _CYCLUS_ template file using `jinja2`.

### Demonstration
__Read fleetcomp reactor data__

In [11]:
pris = import_csv('../../database/reactors_pris_2016.csv', ',')

print(*pris[1:6], sep='\n')
print('.', '.', '.', sep='\n')

__Select reactors in the specified region__

PRIS reactor spreadsheet file contains information about nuclear reactors from various nations. In order to obtain commercial reactors deployed in the United States, a separate function was written that searches for reactors from the US.

In [13]:
def select_region(in_list, region):
    """ Returns a list of reactors that will be deployed for
    CYCLUS by checking the capacity and commercial date

    Parameters
    ----------
    in_list: list
            imported csv file in list format
    region: str
            name of the region

    Returns
    -------
    reactor_list: list
            list of reactors from PRIS
    """
    ASIA = {'IRAN', 'JAPAN', 'KAZAKHSTAN',
            'BANGLADESH', 'CHINA', 'INDIA',
            'UNITED ARAB EMIRATES', 'VIETNAM',
            'PAKISTAN', 'PHILIPPINES', 'SOUTH KOREA'
            }
    UNITED_STATES = {'UNITED STATES'}
    SOUTH_AMERICA = {'ARGENTINA', 'BRAZIL'}
    NORTH_AMERICA = {'CANADA', 'MEXICO', 'UNITED STATES'}
    EUROPE = {'UKRAINE', 'UNITED KINGDOM',
              'POLAND', 'ROMANIA', 'RUSSIA',
              'BELARUS', 'BELGIUM', 'BULGARIA',
              'GERMANY', 'ITALY', 'NETHERLANDS',
              'SWEDEN', 'SWITZERLAND', 'TURKEY',
              'SLOVENIA', 'SOVIET UNION', 'SPAIN',
              'CZECHOSLOVAKIA', 'FINLAND', 'FRANCE'
              }
    AFRICA = {'EGYPT', 'MOROCCO', 'SOUTH AFRICA', 'TUNISIA'}
    ALL = (SOUTH_AMERICA | NORTH_AMERICA |
           EUROPE | ASIA | AFRICA | UNITED_STATES)
    regions = {'ASIA': ASIA,
               'AFRICA': AFRICA,
               'EUROPE': EUROPE,
               'SOUTH_AMERICA': SOUTH_AMERICA,
               'NORTH_AMERICA': NORTH_AMERICA,
               'UNITED_STATES': UNITED_STATES,
               'ALL': ALL}
    if region.upper() not in regions.keys():
        raise ValueError(region + 'is not a valid region')
    reactor_list = []
    for row in in_list:
        country = row[0]
        if country.upper() in regions[region.upper()]:
            capacity = row[3]
            start_date = row[10]
            if confirm_deployment(start_date, capacity):
                reactor_list.append(row)
    return reactor_list


def confirm_deployment(date_str, capacity):
    """ Confirms if reactor is to be deployed for CYCLUS by
    checking if the capacity > 400 and if the commercial date
    is a proper date format.

    Parameters
    ----------
    date_str: str
            the commercial date string from PRIS data file
    capacity: str
            capacity in MWe from RPIS data file

    Returns
    -------
    is_deployed: bool
            determines whether the reactor will be deployed
            in CYCLUS
    """
    is_deployed = False
    if len(date_str) > 4 and float(capacity) > 400:
        try:
            date.parse(date_str)
            is_deployed = True
        except:
            pass
    return is_deployed


reactor_list = select_region(pris, 'UNITED_STATES')
print(*reactor_list[:4], '.', '.', '.', sep='\n')

__Select and render useful reactor information__

In this step, the following information is obtained from the PRIS spreadsheet: reactor name, type, deployed nation, and capacity. Reactor specifications such as number of assemblies, assemblies per batch, mass of core, and mass of assembly were obtained from published sources [1], [2]. Then, `write_reactors` function is used to render these information. The function loops over each row and checks the type of reactor and its capacity. Reactors with capacity less than 400 MWe were assumed to be experimental reactors. Then, all the raw and derived variables necessary for simulation are rendered to individual reactor files.

In [20]:
def write_reactors(in_list, out_path, reactor_template,cycle_time,refuel_time):
    """ Renders CYCAMORE::reactor specifications using jinja2.

    Parameters
    ----------
    in_list: list
        list containing PRIS data
    out_path: str
        output path for reactor files
    reactor_template: str
        path to reactor template

    Returns
    -------
    null
        writes xml files with CYCAMORE::reactor config
    """
    if out_path[-1] != '/':
        out_path += '/'
    pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)
    reactor_template = load_template(reactor_template)
    for row in in_list:
        capacity = float(row[3])
        if capacity >= 400:
            name = row[1].replace(' ', '_')
            assem_per_batch = 0
            assem_no = 0
            assem_size = 0
            reactor_type = row[2]
            latitude = row[13] if row[13] != '' else 0
            longitude = row[14] if row[14] != '' else 0
            if reactor_type in ['BWR', 'ESBWR']:
                assem_no = 732
                assem_per_batch = int(assem_no / 3)
                assem_size = 138000 / assem_no
            elif reactor_type in ['GCR', 'HWGCR']:  # Need batch number
                assem_no = 324
                assem_per_batch = int(assem_no / 3)
                assem_size = 114000 / assem_no
            elif reactor_type == 'HTGR':  # Need batch number
                assem_no = 3944
                assem_per_batch = int(assem_no / 3)
                assem_size = 39000 / assem_no
            elif reactor_type == 'PHWR':
                assem_no = 390
                assem_per_batch = int(assem_no / 45)
                assem_size = 80000 / assem_no
            elif reactor_type == 'VVER':  # Need batch number
                assem_no = 312
                assem_per_batch = int(assem_no / 3)
                assem_size = 41500 / assem_no
            elif reactor_type == 'VVER-1200':  # Need batch number
                assem_no = 163
                assem_per_batch = int(assem_no / 3)
                assem_size = 80000 / assem_no
            else:
                assem_no = 241
                assem_per_batch = int(assem_no / 3)
                assem_size = 103000 / assem_no
            config = reactor_template.render(name=name,
                                             lifetime=get_lifetime(row),
                                             cycletime = cycle_time, 
                                             refueltime = refuel_time,
                                             assem_size=assem_size,
                                             n_assem_core=assem_no,
                                             n_assem_batch=assem_per_batch,
                                             power_cap=row[3],
                                             lon=longitude,
                                             lat=latitude)
            with open(out_path + name.replace(' ', '_') + '.xml',
                      'w') as output:
                output.write(config)


def get_lifetime(in_row):
    """ Calculates the lifetime of a reactor using first
    commercial date and shutdown date. Defaults to 720 months
    if shutdown date is not available.

    Parameters
    ----------
    in_row: list
        single row from PRIS data that contains reactor
        information

    Returns
    -------
    lifetime: int
        lifetime of reactor
    """
    comm_date = in_row[10]
    shutdown_date = in_row[11]
    if not shutdown_date.strip():
        return 720
    else:
        n_days_month = 365.0 / 12
        delta = (date.parse(shutdown_date) - date.parse(comm_date)).days
        return int(delta / n_days_month)


out_path = 'cyclus/input/' + region + '/reactors'
reactor_template = 'templates/reactors_template.xml'
write_reactors(reactor_list, out_path, reactor_template,18,1) # change cycle and refuel time 

[reactors](cyclus/input/united_states/reactors) folder contains individual reactor specifications.

## Writing Deployment

In order to ensure proper deployment of all reactors during simulation, a function was written to calculate then save deployment information. `build_time`, the number of timesteps taken to deploy the reactors, is calculated using the difference between simulation start date and the reactor first grid date in _CYCLUS_ timesteps (months).

### Demonstration 
__Read PRIS reactor data__

`deploy_reactors` function is used for this application. The function imports the PRIS reactor file, produces the folder structure necessary for file output, and calls `get_buildtime` function and `write_deployment` function. `get_buildtime` function parses the deployment dates stored in the PRIS reactor file, and calculates the difference between simulation start date and reactor deployment date in months. Then, the resulting build_time is stored in a dictionary object with the reactor name as key, and the deployed country and deployment month as value. Finally, `write_deployment` function uses the dictionary obtained from `get_buildtime` function to produce deployment specifications.

In [15]:
def deploy_reactors(in_csv, region, start_year, deployinst_template,
                    inclusions_template, reactors_path, deployment_path):
    """ Generates xml files that specify the reactors that will be included
    in a CYCLUS simulation.

    Parameters
    ---------
    in_csv: str
        path to pris reactor database
    region: str
        region name
    start_year: int
        starting year of simulation
    deployinst_template: str
        path to deployinst template
    inclusions_template: str
        path to inclusions template
    reactors_path: str
        path containing reactor files
    deployment_path: str
        output path for deployinst xml

    Returns
    -------
    buildtime_dict: dict
        dictionary with key=[name of reactor], and
        value=[set of country and buildtime]
    """
    lists = []
    if reactors_path[-1] != '/':
        reactors_path += '/'
    for files in os.listdir(reactors_path):
        lists.append(reactors_path + files)
    in_data = import_csv(in_csv, ',')
    reactor_list = select_region(in_data, region)
    buildtime = get_buildtime(reactor_list, start_year, lists)
    write_deployment(buildtime, deployment_path, deployinst_template,
                     inclusions_template)
    return buildtime


def get_buildtime(in_list, start_year, path_list):
    """ Calculates the buildtime required for reactor
    deployment in months.

    Parameters
    ----------
    in_list: list
        list of reactors
    start_year: int
        starting year of simulation
    path_list: list
        list of paths to reactor files

    Returns
    -------
    buildtime_dict: dict
        dictionary with key=[name of reactor], and
        value=[set of country and buildtime]
    """
    buildtime_dict = {}
    for row in in_list:
        comm_date = date.parse(row[10])
        start_date = [comm_date.year, comm_date.month, comm_date.day]
        delta = ((start_date[0] - int(start_year)) * 12 +
                 (start_date[1]) +
                 round(start_date[2] / (365.0 / 12)))
        for index, reactor in enumerate(path_list):
            name = row[1].replace(' ', '_')
            country = row[0]
            file_name = (reactor.replace(
                os.path.dirname(path_list[index]), '')).replace('/', '')
            if (name + '.xml' == file_name):
                buildtime_dict.update({name: (country, delta)})
    return buildtime_dict


def write_deployment(in_dict, out_path, deployinst_template,
                     inclusions_template):
    """ Renders jinja template using dictionary of reactor name and buildtime.
    Outputs an xml file that uses xinclude to include the reactor xml files
    located in cyclus_input/reactors.

    Parameters
    ---------
    in_dict: dictionary
        dictionary with key=[reactor name], and value=[buildtime]
    out_path: str
        output path for files
    deployinst_template: str
        path to deployinst template
    inclusions_template: str
        path to inclusions template

    Returns
    -------
    null
        generates input files that have deployment and xml inclusions
    """
    if out_path[-1] != '/':
        out_path += '/'
    pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)
    deployinst_template = load_template(deployinst_template)
    inclusions_template = load_template(inclusions_template)
    country_list = {value[0] for value in in_dict.values()}
    for nation in country_list:
        temp_dict = {}
        for reactor in in_dict.keys():
            if in_dict[reactor][0].upper() == nation.upper():
                temp_dict.update({reactor: in_dict[reactor][1]})
        pathlib.Path(out_path + nation.replace(' ', '_') +
                     '/').mkdir(parents=True, exist_ok=True)
        deployinst = deployinst_template.render(reactors=temp_dict)
        with open(out_path + nation.replace(' ', '_') +
                  '/deployinst.xml', 'w') as output1:
            output1.write(deployinst)
    inclusions = inclusions_template.render(reactors=in_dict)
    with open(out_path + 'inclusions.xml', 'w') as output2:
        output2.write(inclusions)


pris_file = '../../database/reactors_pris_2016.csv'
deployinst_tmpl = 'templates/' + region + '/deployinst_template.xml'
inclusions_tmpl = 'templates/inclusions_template.xml'
reactor_path = 'cyclus/input/' + region + '/reactors'
dployment_path = 'cyclus/input/' + region + '/buildtimes'
print(reactor_path)
buildtime = deploy_reactors(pris_file, region, 1965, deployinst_tmpl,
                            inclusions_tmpl, reactor_path, dployment_path)

for k in list(buildtime.keys())[:4]:
    print(k, ': ', buildtime[k])
print('.', '.', '.', sep='\n')

[deployinst folder](cyclus/input/united_states/buildtimes/) and [inclusions.xml](cyclus/input/united_states/buildtimes/inclusions.xml) show the rendered result. 

`deployinst.xml` contains configurations for _CYCAMORE::DeployInst_ archetype for the agents in the deploy institution and `inclusions.xml` contains xml inclusions so that the final _CYCLUS_ input file references the individual reactor files produced during __Obtaining Deployment Data__ step.

## XML inclusion and  Setting XML base for final input file

For this investigation, `XInclude` was used to produce modular _CYCLUS_ input files. Rather than a single lengthy input file, the use of xml inclusion allows the creation of _CYCLUS_ input files that contain certain sections such as deployment information, reactor specifications, or even entire regions. This introduces modularity to _CYCLUS_ input files and simulations. To use `XInclude` to reference other files within the input file, an xml base needs to be specified. 

__XInclude basics__

In order to use `XInclude` simply add the `XInclude` namespace to the root element of the xml file that needs to reference another xml file.

> xmlns:xi="http://www.w3.org/2001/XInclude"

Then, reference the xml file that needs to be added to the base xml file using an `href` tag under the `XInclude` namespace.

> `<xi:include href="link_to_xml_file_to_be_added.xml" />`


__Example__

Below is an example of xml inclusion from the final _CYCLUS_ [input file](cyclus/input/united_states.xml) (click to open the full file).

```
<simulation xml:base="/home/gyutae/cyclus/predicting-the-past/cyclus/input/"
  xmlns:xi="http://www.w3.org/2001/XInclude">
  <control>
    <duration>1020</duration>
    <startmonth>1</startmonth>
    <startyear>1965</startyear>
  </control>
  <archetypes>
    <spec><lib>cycamore</lib> <name>Enrichment</name> </spec>
    <spec><lib>cycamore</lib> <name>Reactor</name>    </spec>
    <spec><lib>cycamore</lib> <name>Sink</name>       </spec>
    <spec><lib>cycamore</lib> <name>Source</name>     </spec>
    <spec><lib>cycamore</lib> <name>Storage</name>    </spec>
    <spec><lib>cycamore</lib> <name>DeployInst</name> </spec>
    <spec><lib>cycamore</lib> <name>ManagerInst</name></spec>
    <spec><lib>agents</lib>   <name>NullRegion</name> </spec>
  </archetypes>
  <xi:include href="UNITED_STATES/buildtimes/inclusions.xml#xpointer(/inclusions/child::*)"/>
.
.
.
```
As shown above, `XInclude` namespace was added to the root element: `simulation`. The xml file to be added was declared under the namespace using `xi:include` with the relative link `UNITED_STATES/buildtimes/inclusions.xml#xpointer(/inclusions/child::*)`.

__Rationale for setting xml base__

One issue that arises with the use of relative path to reference an external document, is that the xml parser (including the one used in _CYCLUS_) does not know where to find the referenced documents. Thus, parsing the final input file from different paths yielded different results and often lead to _unable-to-find-external-entity_ errors. Setting an xml base allows the parser to correctly find the referenced files.

__Rendering final input file__

Setting an xml base is done by finding the absolute path of the _CYCLUS_ input file, and using `jinja2` to render the absolute path.

In [29]:
def render_cyclus(cyclus_template, region, in_dict, out_path,burn_up):
    """ Renders final CYCLUS input file with xml base, and institutions
    for each country

    Parameters
    ----------
    cyclus_template: str
        path to CYCLUS input file template
    region: str
        region chosen for CYCLUS simulation
    in_dict: dictionary
        in_dict should be buildtime_dict from get_buildtime function
    out_path: str
        output path for CYCLUS input file
    output_name:

    Returns
    -------
    null
        writes CYCLUS input file in out_path
    """
    if out_path[-1] != '/':
        out_path += '/'
    cyclus_template = load_template(cyclus_template)
    country_list = {value[0].replace(' ', '_') for value in in_dict.values()}
    rendered = cyclus_template.render(burnup = burn_up, 
                                      countries=country_list,
                                      base_dir=os.path.abspath(out_path) + '/')
    with open(out_path + region + '.xml', 'w') as output:
        output.write(rendered)


cyclus_tmpl = ('templates/' + region + '/' + region + '_template.xml')
render_cyclus(cyclus_tmpl, region, buildtime, 'cyclus/input/',51) # change burn up 

[united_states.xml](cyclus/input/united_states.xml) shows the final _CYCLUS_ input file with all the xml inclusions and xml base.

__Running _CYCLUS_ Simulation__

Using the final input file above, a _CYCLUS_ simulation is run using the following command on bash. The output of the simulation is uploaded to the ARFC Box for use if _CYCLUS_ is not installed in the device. In such case, skip to _Analysis and Results_ section.

In [ ]:
!rm cyclus/united_states.sqlite
!cyclus -i ./cyclus/input/united_states.xml -o ./cyclus/united_states.sqlite --warn-limit 0

The first command removes the previous simulation output file. This is to prevent the final output file from having the results of any previous simulations. The simulation results are stored as an SQLite file [united_states.sqlite](cyclus/united_states.sqlite). A set of functions were written in python to obtain relevant data from the output file. With python, the results of the queries can be processed to perform meaningful analysis.

## Analysis and Results

The results for the analysis performed above are shown below. Unfortunately, the only data that was available for comparison at the time of writing was power generated over time published by the Nuclear Energy Institute (NEI) [4]. The NEI publication  contains power generated by US commercial nuclear power plants from 1971 to 2016. A similar plot is generated from the simulation results to compare the power generated and is shown in Figure 10. The plot of power generated from the data published by the NEI is shown in Figure 11.

__Connect to SQLite file__

To make sqlite queries in python, a cursor, which acts like a pointer to the sqlite file is required. The use of `row_factory` class under `sqlite3` library allows the sqlite table to be indexed by integer values and by case-insensitive row name. `row_factory` produces readable code, and allows easy debugging.

In [ ]:
def get_cursor(file_name):
    """ Connects and returns a cursor to an sqlite output file

    Parameters
    ----------
    file_name: str
        name of the sqlite file

    Returns
    -------
    sqlite cursor
    """
    con = lite.connect(file_name)
    con.row_factory = lite.Row
    return con.cursor()


cursor = get_cursor('cyclus/united_states.sqlite')

__Get simulation start time, duration, and timestep__

All analysis results are plotted for a visual representation. `get_timesteps` function is used to obtain simulation start time, and timestep, which are required for plotting.

In [ ]:
def get_timesteps(cur):
    """ Returns simulation start year, month, duration and
    timesteps (in numpy linspace).

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    init_year: int
        start year of simulation
    init_month: int
        start month of simulation
    duration: int
        duration of simulation
    timestep: list
        linspace up to duration
    """
    info = cur.execute('SELECT initialyear, initialmonth, '
                       'duration FROM info').fetchone()
    init_year = info['initialyear']
    init_month = info['initialmonth']
    duration = info['duration']
    timestep = np.linspace(0, duration - 1, num=duration)

    return init_year, init_month, duration, timestep


ini_yr, ini_month, dur, timestep = get_timesteps(cursor)
print('Year: ', ini_yr, '\nMonth: ', ini_month, '\nDuration: ',
      dur, '\nTimestep: ', timestep)

__Nat_u consumption vs Time__

`nat_u_timeseries` function returns a timeseries list of natural uranium sent to the enrichment facility. Unfortunately, this does not represent the amount of natural uranium used in real life as _CYCAMORE::enrichment_ facility does not enrich fuel on a need-basis. Due to current limits in the software, the enrichment facility enriches the same amount of natural uranium at each timestep. When _CYCAMORE::enrichment_ is updated to enrich fuel on a need-basis, this analysis would be more accurate. 

The `nat_u_timeseries` function makes an sqlite query on the `timeseriesenrichmentfeed` table from the simulation output file and passes the results to `get_timeseries_cum` function. The `get_timeseries_cum` function then calculates the chronological cumulative sum of the natural uranium transferred. The resulting timeseries list is used to plot the cumulative amount of natural uranium used in enrichment over time. 

In [ ]:
def nat_u_timeseries(cur):
    """ Finds natural uranium supply from source
        Since currently the source supplies all its capacity,
        the timeseriesenrichmentfeed is used.

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    get_timeseries: function
        calls a function that returns timeseries list of natural U
        demand from enrichment [MTHM]
    """
    init_year, init_month, duration, timestep = get_timesteps(cur)

    # Get Nat U feed to enrichment from timeseriesenrichmentfeed
    feed = cur.execute('SELECT time, sum(value) '
                       'FROM timeseriesenrichmentfeed '
                       'GROUP BY time').fetchall()
    return get_timeseries_cum(feed, duration, True)


def get_timeseries_cum(in_list, duration, kg_to_tons):
    """ returns a timeseries list from in_list data.

    Parameters
    ----------
    in_list: list
        list of data to be created into timeseries
        list[0] = time
        list[1] = value, quantity
    multiplyby: int
        integer to multiply the value in the list by for
        unit conversion from kilograms
    kg_to_tons: bool
        if True, list returned has units of tons
        if False, list returned as units of kilograms

    Returns
    -------
    timeseries of commodities in kg or tons
    """
    value = 0
    value_timeseries = []
    array = np.array(in_list)
    if len(in_list) > 0:
        for i in range(0, duration):
            value += sum(array[array[:, 0] == i][:, 1])
            if kg_to_tons:
                value_timeseries.append(value * 0.001)
            else:
                value_timeseries.append(value)
    return value_timeseries


img_out = 'analysis/results/' + region + '/'
pathlib.Path(img_out).mkdir(parents=True, exist_ok=True)
name = 'Nat_u_consumed vs time'
consumption = {'Nat_u_consumption': nat_u_timeseries(cursor)}
an.stacked_bar_chart(consumption, timestep,
                     'Time [Yr]', 'Nat_u_Consumed [MTHM]',
                     name, img_out + name, ini_yr)

Image(filename=img_out + name + '.png')

As shown in the figure above, natural uranium consumed over the period of the simulation linearly increases. This demonstrates the constant rate of fuel processing performed by the enrichment facility regardless of the demand for fresh fuel. This will hopefully be improved in the future so that the enrichment facilities take the demand for fresh fuel into account.

__Amount of fuel into reactors vs. time__

A more accurate analysis for fuel consumption is the amount of fuel sent to reactors over time. While the enrichment facility constantly produces fresh fuel, the total amount of fuel sent to reactors changes based on the number and size of the reactors in operation at each timestep. This analysis is performed with `fuel_into_reactors` function. The function makes an sqlite query to the `resources`, `transactions`, and `agententry` table to obtain the amount of fuel sent to reactors throughout the simulation. The resulting data was sent to `get_timeseries_cum` function to obtain a timeseries list of different types of fuel sent to reactors throughout the simulation.

In [ ]:
def fuel_into_reactors(cur):
    """ Finds timeseries of mass of fuel received by reactors

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    timeseries list of fuel into reactors [tons]
    """
    init_year, init_month, duration, timestep = get_timesteps(cur)
    fuel = cur.execute('SELECT time, sum(quantity) FROM transactions '
                       'INNER JOIN resources ON '
                       'resources.resourceid = transactions.resourceid '
                       'INNER JOIN agententry ON '
                       'transactions.receiverid = agententry.agentid '
                       'WHERE spec LIKE "%Reactor%" '
                       'GROUP BY time').fetchall()

    return get_timeseries_cum(fuel, duration, True)


name = 'Fuel into Reactors over Time'
to_reactor = {'Fuel_to_reactor': fuel_into_reactors(cursor)}
an.stacked_bar_chart(to_reactor, timestep,
                     'Time [Yr]', 'Fuel into Reactors [MTHM]',
                     name, img_out + name, ini_yr)

Image(filename=img_out + name + '.png')

As shown in the figure above, the fuel into reactors over time is not linear and changes with fuel demand of the reactors. This figure reflectes the state of the reactors (whether they were operating or refueling), and the number of reactors operating within the simulation. From the plot, the number of reactors greatly increased around 1990s.

__Fuel utilization vs. time__

Fuel utilization factor is the ratio of fuel spent to the amount of natural uranium consumed. While, fuel utilization factor is a value of interest, it is inaccurate at the moment of this investigation because the enrichment facilities in _CYCLUS_ does not perform a demand-driven fuel enrichment. Since the amount of natural uranium consumed is not accurate, fuel utilization factor, a derived variable, is also inaccurate. However, the functions used to calculate the fuel utilization factor over time is displayed for future usage. This is performed with `u_util_calc` function, which runs `nat_u_timeseries` function and `fuel_into_reactors` function to obtain the timeseries lists of natural uranium consumed and the amount of fuel consumed. Then, and element-wise division of the two lists were performed to obtain the fuel utilization factor.

In [ ]:
def u_util_calc(cur):
    """ Returns fuel utilization factor of fuel cycle

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    u_util_timeseries: numpy array
        Timeseries of Uranium utilization factor
    Prints simulation average Uranium Utilization
    """
    # timeseries of natural uranium
    u_supply_timeseries = np.array(nat_u_timeseries(cur))

    # timeseries of fuel into reactors
    fuel_timeseries = np.array(fuel_into_reactors(cur))

    # timeseries of Uranium utilization
    u_util_timeseries = np.nan_to_num(fuel_timeseries / u_supply_timeseries)
    print('The Average Fuel Utilization Factor is: ')
    print(sum(u_util_timeseries) / len(u_util_timeseries))

    return u_util_timeseries


name = 'Fuel utilization'
fuel_util = {'Fuel Utilization Factor': u_util_calc(cursor)}
an.stacked_bar_chart(fuel_util, timestep,
                     'Time [Yr]', 'Fuel utiliization',
                     name, img_out + name, ini_yr)

Image(filename=img_out + name + '.png')

__Capacity vs. time__

Total capacity over time is obtained using `get_power_dict` function and `capacity_calc` function. The `get_power_dict` function performs four sqlite queries to obtain the following: simulation start date, timesteps, institutions declared in _CYCLUS_, and reactors entry and exit timesteps. The results are sent to `capacity_calc` function, which calculates the timeseries value of the total capacity of the reactors in each institution.

In [ ]:
def get_power_dict(cur):
    """ Gets dictionary of power capacity by calling capacity_calc

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    ------
    power_dict: dictionary
        "dictionary with key=government, and
        value=timeseries list of installed capacity"
    """
    init_year, init_month, duration, timestep = get_timesteps(cur)
    governments = get_inst(cur)

    # get power cap values
    entry = cur.execute('SELECT max(value), timeseriespower.agentid, '
                        'parentid, entertime FROM agententry '
                        'INNER JOIN timeseriespower '
                        'ON agententry.agentid = timeseriespower.agentid '
                        'GROUP BY timeseriespower.agentid').fetchall()

    exit_step = cur.execute('SELECT max(value), timeseriespower.agentid, '
                            'parentid, exittime FROM agentexit '
                            'INNER JOIN timeseriespower '
                            'ON agentexit.agentid = timeseriespower.agentid'
                            ' INNER JOIN agententry '
                            'ON agentexit.agentid = agententry.agentid '
                            'GROUP BY timeseriespower.agentid').fetchall()

    return capacity_calc(governments, timestep, entry, exit_step)


def get_inst(cur):
    """ Returns prototype and agentids of institutions

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    -------
    sqlite query result (list of tuples)
    """
    return cur.execute('SELECT prototype, agentid FROM agententry '
                       'WHERE kind = "Inst"').fetchall()


def capacity_calc(governments, timestep, entry, exit_step):
    """Adds and subtracts capacity over time for plotting

    Parameters
    ----------
    governments: list
        list of governments (countries)
    timestep: np.linspace
        list of timestep from 0 to simulation time
    entry: list
        power_cap, agentid, parentid, entertime
        of all entered reactors
    exit_step: list
        power_cap, agentid, parenitd, exittime
        of all decommissioned reactors

    Returns
    -------
    power_dict: dictionary
        "dictionary with key=government, and
        value=timeseries list capacity"
    """
    power_dict = collections.OrderedDict()
    for gov in governments:
        capacity = []
        cap = 0
        for t in timestep:
            for enter in entry:
                if (enter['entertime'] == t and
                        enter['parentid'] == gov['agentid']):
                    cap += enter['max(value)'] * 0.001
            for dec in exit_step:
                if (dec['exittime'] == t and
                        dec['parentid'] == gov['agentid']):
                    cap -= dec['max(value)'] * 0.001
            capacity.append(cap)
        power_dict[gov['prototype']] = np.asarray(capacity)

    return power_dict


name = 'Net Capacity vs Time'
capacity_dict = get_power_dict(cursor)
an.stacked_bar_chart(capacity_dict, timestep,
                     'Years', 'Net_Capacity [GWe]',
                     name, img_out + name, ini_yr)

Image(filename=img_out + name + '.png')

As shown in the figure above, the net capacity greatly increases between 1965 and 1990. The increase in capacity reflects the increase in the number of reactors in operation. Then, the capacity decreases gradually from around 1995. This trend is also shown in the figure of number of reactors over time below.

__Number of reactors vs time__

A very similar procedure is used to obtain the number of reactors over time. The same four queries made to calculate capacity over time are made to get the number of reactors over time. Then, the result of the queries is passed to `reactor_deployments` function, which calculates the number of reactors operating over time. 

In [ ]:
def get_deployment_dict(cur):
    """ Gets dictionary of reactors deployed over time
    by calling reactor_deployments

    Parameters
    ----------
    cur: sqlite cursor
        sqlite cursor

    Returns
    ------
    num_dict: dictionary
        "dictionary with key=government, and
        value=timeseries list of number of reactors"
    """
    init_year, init_month, duration, timestep = get_timesteps(cur)
    governments = get_inst(cur)

    # get power cap values
    entry = cur.execute('SELECT max(value), timeseriespower.agentid, '
                        'parentid, entertime FROM agententry '
                        'INNER JOIN timeseriespower '
                        'ON agententry.agentid = timeseriespower.agentid '
                        'GROUP BY timeseriespower.agentid').fetchall()

    exit_step = cur.execute('SELECT max(value), timeseriespower.agentid, '
                            'parentid, exittime FROM agentexit '
                            'INNER JOIN timeseriespower '
                            'ON agentexit.agentid = timeseriespower.agentid'
                            ' INNER JOIN agententry '
                            'ON agentexit.agentid = agententry.agentid '
                            'GROUP BY timeseriespower.agentid').fetchall()

    return reactor_deployments(governments, timestep, entry, exit_step)


def reactor_deployments(governments, timestep, entry, exit_step):
    """Adds and subtracts number of reactors deployed over time
    for plotting

    Parameters
    ----------
    governments: list
        list of governments (countries)
    timestep: np.linspace
        list of timestep from 0 to simulation time
    entry: list
        power_cap, agentid, parentid, entertime
        of all entered reactors

    exit_step: list
        power_cap, agentid, parenitd, exittime
        of all decommissioned reactors

    Returns
    -------
    deployment: dictionary
        "dictionary with key=government, and
        value=timeseries number of reactors"
    """
    deployment = collections.OrderedDict()
    for gov in governments:
        num_reactors = []
        count = 0
        for t in timestep:
            for enter in entry:
                if (enter['entertime'] == t and
                        enter['parentid'] == gov['agentid']):
                    count += 1
            for dec in exit_step:
                if (dec['exittime'] == t and
                        dec['parentid'] == gov['agentid']):
                    count -= 1
            num_reactors.append(count)
        deployment[gov['prototype']] = np.asarray(num_reactors)

    return deployment


name = 'Number of Reactors vs Time'
an.stacked_bar_chart(get_deployment_dict(cursor),
                     timestep, 'Years', 'Number of Reactors',
                     name, img_out + name, ini_yr)

Image(filename=img_out + name + '.png')

__Power generated vs. time__

The amount of power generated over time is calculated from capacity over time with a few assumptions. Power generated is calculated from capacity with the the following function.

$$P(t)=C(t)\times CF(t)\times HRPYR$$

where,

$$P(t) = Power\ generated\ over\ a\ year\ [GWh]$$
$$C(t) = Capacity\ over\ time\ [GWe]$$
$$CF(t)= Capacity\ factor$$
$$HRPYR= Number\ of\ hours\ in\ a\ year$$

The following assumptions are used for the calculation above.

1. Refueling time of all reactors is 1 month.
2. Capacity factor is constant throughout the year, and identical for all reactors.
3. Reactors generate their full power even during startup and shutdown.

Capacity factor data is obtained from the Energy Information Administration (EIA), which has the yearly average capacity factor for reactors in the United States from 1971 to 2017. The capacity factor is assumed constant from 2016 to the end of the simulation and from 1965 to 1971. Capacity factor for each year was stored in a separate [spreadsheet file](analysis/published_data/US/capacity_factor_extrapolated.csv). A set of python functions are used to calculate the power generated over time.

In [ ]:
hours_in_year = 24 * 365.25
cf_data = import_csv('analysis/published_data/united_states/capacity_factor_extrapolated.csv',
                     ',')
print(*cf_data[:8], sep='\n')
print('.', '.', '.', sep='\n')

The capacity factor is assumed to be constant at 48.2% from 1965 to 1971, and at 92.5% from 2017 to 2049. Power generated at each timestep is calculated by performing an element-wise multiplication of  the capacity factor data and the capacity. However, since capacity factor is stored for each year, Assumption 2 is used. A separate function is used to create a list that stores the capacity factor for each timestep of the simulation.

In [ ]:
def get_cf(in_list):
    """ Creates a list of capacity factor from
    the imported csv file

    Parameters
    ----------
    in_list: list
        list containing data stored in a csv file

    Returns
    -------
    cf: list
        list of capacity factors per month
    """
    cf = []
    for row in in_list[1:]:
        for i in range(0, 12):
            cf.append(float(row[1]) / 100)

    return cf

name = 'Power Generated'
cf = np.asarray(get_cf(cf_data))
capacity = capacity_dict['United_States']
generated = {"United_States": np.multiply(capacity, cf) * hours_in_year}
an.stacked_bar_chart(generated, timestep,
                     'Years', 'Power Generated [GWh]',
                     name, img_out + name, ini_yr)

Image(filename=img_out + name + '.png')

__Power generated between 1971 and 2016__

The NEI published the amount of power generated by nuclear power plants in the United States between 1971 and 2016 [4]. This information is used to compare the simulation results with the actual data. The data obtained from the previous analysis was used to obtain the power generated between 1971 and 2016 by calculating the starting index and the ending index. The published power generation was plotted for comparison.

In [ ]:
start_year = 1971
end_year = 2017
start_index = (start_year - 1965) * 12
end_index = (end_year - 1965) * 12
dura = end_year - start_year

name = 'Power Generated 1971~2016'
timestep_range = timestep[start_index: end_index]
cf_range = cf[start_index: end_index]
generated_range = {"United_States":
                   generated["United_States"][start_index: end_index]}
an.stacked_bar_chart(generated_range, timestep_range,
                     "Years", "Power Generated [GWh]",
                     name, img_out + name, 1965)

Image(filename=img_out + name + '.png')

In [ ]:
Image(filename='analysis/published_data/united_states/Power Generated NEI.png')

As shown in the two plots above,the plot of power generated between 1971 and 2016 from the _CYCLUS_ simulation matches  closely to that published by the NEI. There are a few differences between the two plots. First, the power generated according to _CYCLUS_ is slightly higher overall than that according to NEI. This can be explained from the difference in simulation and real-world refueling times. 

Due to software limitations of _CYCLUS_, refueling time for reactors can only be entered in integers of month, and stayed constant throughout the lifetime of the plant whereas actual refueling time varies in floating points of month and changes depending on various factors. The _CYCLUS_ simulation used to obtain the power generated over time assumes a refueling period of 1 month. According to the NEI, the average  refueling period for reactors in the United States varies quite significantly. The average refueling period in 1990 was 104 days, a period of over 3 months, and generally decreases to an average refueling period of 35 days, a period  just over 1 month, in 2017. As published by the NEI, the average refueling period of nuclear reactors in the United States is greater than 1 month between 1990 and 2017 [3]. This means that the power generated by the reactors deployed in the _CYCLUS_ simulation will have a greater amount of power generated over a period of time.

## Conclusion

To conclude, _CYCLUS_ performed simulations at an accurate level as shown in the comparison of the power generation plots. However, some improvements in _CYCLUS_ would be benefitial in producing a more accurate simulation: the implementation of an enrichment facility that processes fuel on a need basis. There are other improvements that can be potentially made for this simulation to improve its accuracy.

+  Specify actual burnup for reactors
    + Currently, the reactors deployed in Cyclus are all deployed with a burnup of 51 GWd/MTHM. 

### References

[1] N. Todreas and M. Kazimi, Nuclear systems. Boca Raton, FL: CRC Press, 2012.


[2] D. Cacuci, Handbook of nuclear engineering. New York: Springer, 2010.


[3] US Nuclear Refueling Outage Days. (n.d.). Retrieved from https://www.nei.org/Knowledge-Center/Nuclear-Statistics/US-Nuclear-Power-Plants/US-Nuclear-Refueling-Outage-Days


[4] US Nuclear Generating Statistics. (n.d.). Retrieved from https://www.nei.org/Knowledge-Center/Nuclear-Statistics/US-Nuclear-Power-Plants/US-Nuclear-Generating-Statistics